In [18]:
from function import load_data, get_standardized_data, get_nav_adjusted

Welcome to use Wind Quant API for Python (WindPy)!

COPYRIGHT (C) 2024 WIND INFORMATION CO., LTD. ALL RIGHTS RESERVED.
IN NO CIRCUMSTANCE SHALL WIND BE RESPONSIBLE FOR ANY DAMAGES OR LOSSES CAUSED BY USING WIND QUANT API FOR Python.


In [19]:
data_path = "data\旌安1号_2025-02-07.csv"
data = load_data(data_path)
data.head()

,日期,单位净值,累计净值
0,2018/6/29,0.999,0.999
1,2018/7/6,1.005,1.005
2,2018/7/13,1.017,1.017
3,2018/7/20,1.034,1.034
4,2018/7/27,1.046,1.046


In [20]:
df = get_standardized_data(data)
df.head()

,date,nav_unit,nav_accumulated
0,2018-06-29,1.000,1.000
1,2018-07-06,1.006,1.006
2,2018-07-13,1.018,1.018
3,2018-07-20,1.035,1.035
4,2018-07-27,1.047,1.047


In [21]:
df = get_nav_adjusted(df)
df.head()

,date,nav_unit,nav_accumulated,nav_adjusted
0,2018-06-29,1.000,1.000,1.000
1,2018-07-06,1.006,1.006,1.006
2,2018-07-13,1.018,1.018,1.018
3,2018-07-20,1.035,1.035,1.035
4,2018-07-27,1.047,1.047,1.047
